In [1]:
%matplotlib inline
import matplotlib.pylab as plt
import IPython.display as ipd
import numpy as np
import pandas as pd
import scipy
import sklearn
import librosa
import librosa.display
from ast import literal_eval
from joblib import Parallel, delayed
from tqdm import tqdm
import seaborn as sns
import pickle
import os.path

np.random.seed(37)

In [2]:
recording_df = pd.read_csv('recording.csv', converters={'types': literal_eval})

In [3]:
recording_df.head(5)

,id,state,en,sp,ssp,gen,lat,lng,types,mp3,mp3_size,duration
0,408991,michigan,black-capped chickadee,atricapillus,atricapillus,poecile,42.1696,-84.2051,[call],./mp3/408991.mp3,1056878,26.319550
1,408988,michigan,black-capped chickadee,atricapillus,atricapillus,poecile,42.1696,-84.2051,[call],./mp3/408988.mp3,1718318,42.855550
2,407827,new york,black-capped chickadee,atricapillus,NaN,poecile,42.0330,-79.4222,[song],./mp3/407827.mp3,281570,17.342125
3,406662,illinois,black-capped chickadee,atricapillus,NaN,poecile,41.3260,-89.8926,[song],./mp3/406662.mp3,3878909,96.810275
4,394059,new jersey,black-capped chickadee,atricapillus,NaN,poecile,40.7958,-74.0492,"[alarm call, call, flight call]",./mp3/394059.mp3,154378,7.272000


In [4]:
def get_spectral_features(y, sr):
    centroid = librosa.feature.spectral_centroid(y, sr=sr)
    bandwidth = librosa.feature.spectral_bandwidth(y, sr=sr)
    flatness = librosa.feature.spectral_flatness(y)
    rolloff = librosa.feature.spectral_rolloff(y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    
    return (centroid, bandwidth, flatness, rolloff, zcr)

def get_chroma_features(y, sr, n_mfcc=12):
    mfcc = librosa.feature.mfcc(y, sr=sr, n_mfcc=n_mfcc).T
    stft = librosa.feature.chroma_stft(y, sr=sr).T
    cqt = librosa.feature.chroma_cqt(y, sr=sr, n_chroma=n_mfcc).T
    cens = librosa.feature.chroma_cens(y, sr=sr, n_chroma=n_mfcc).T
    poly = librosa.feature.poly_features(y, sr=sr, order=n_mfcc - 1).T
    
    scaler = sklearn.preprocessing.StandardScaler()
    mfcc = scaler.fit_transform(mfcc)
    
    return np.hstack((mfcc, stft, cqt, cens, poly))

def pickle_data(fname, data):
    pickle.dump(data, open(fname, 'wb'))
    
def save_features(idx, fname, duration=60.0):
    result = 1
    sname = './features/spectral/{}.p'.format(idx)
    cname = './features/chroma/{}.p'.format(idx)
    
    if os.path.exists(sname) is True and os.path.exists(cname) is True:
        return result
    
    try:
        y, sr = librosa.load(fname, duration=duration)

        if os.path.exists(sname) is False:
            try:
                spectral_features = get_spectral_features(y, sr)
                pickle_data(sname, spectral_features)
            except:
                result = -1

        if os.path.exists(cname) is False:
            try:
                chroma_features = get_chroma_features(y, sr)
                pickle_data(cname, chroma_features)
            except:
                result = -2
    except:
        result = -3
    
    return result

In [5]:
results = Parallel(n_jobs=-1, verbose=5)(delayed(save_features)(r['id'], r['mp3']) for i, r in recording_df.iterrows())

[Parallel(n_jobs=-1)]: Done 464 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 5324 tasks      | elapsed:    1.3s


JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/home/super/anaconda3/envs/ds/lib/python2.7/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    169     pkg_name = mod_name.rpartition('.')[0]
    170     main_globals = sys.modules["__main__"].__dict__
    171     if alter_argv:
    172         sys.argv[0] = fname
    173     return _run_code(code, main_globals, None,
--> 174                      "__main__", fname, loader, pkg_name)
        fname = '/home/super/anaconda3/envs/ds/lib/python2.7/site-packages/ipykernel_launcher.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = ''
    175 
    176 def run_module(mod_name, init_globals=None,
    177                run_name=None, alter_sys=False):
    178     """Execute a module's code without importing it

..........................

In [ ]:
print('{} files processed'.format(np.sum(results)))